# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [9]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [7]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("vaccinated.json", "rt") as INFILE:
    vacdata=json.load(INFILE)
with open("pcrtest.json", "rt") as INFILE:
    pcrdata=json.load(INFILE)
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [9]:
def vac_data(rawdata):
    datalist = rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    pcrdf=pd.DataFrame(index=index, columns=['death', 'first', 'second', 'third'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['death', 'first', 'second', 'third']:
            if pd.isna(pcrdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                pcrdf.loc[date, column]=value
    pcrdf.fillna(0.0, inplace=True)
    return pcrdf


def pcr_data(rawdata):
    datalist = rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    pcrdf=pd.DataFrame(index=index, columns=['case', 'pcr', 'hospital'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['case', 'pcr', 'hospital']:
            if pd.isna(pcrdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                pcrdf.loc[date, column]=value
    pcrdf.fillna(0.0, inplace=True)
    return pcrdf



# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
pcrdf = pcr_data(pcrdata) # df is the dataframe for plotting
vacdf = vac_data(vacdata)

In [13]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api(num):
    filter = ['areaType=nation', 'areaName=England']
    structure1 = {
    "date": "date",
    "death": "newDeaths28DaysByDeathDate",
    "first": "newPeopleVaccinatedFirstDoseByPublishDate",
    "second": "newPeopleVaccinatedSecondDoseByPublishDate",
    "third": "newPeopleVaccinatedThirdInjectionByPublishDate"
    }
    structure2 = {
        "date": "date",
        "case": "newCasesBySpecimenDate",
        "pcr": "newPCRTestsByPublishDate",
        "hospital": "newAdmissions",
    }
    if num == 1:
        api = Cov19API(filters=filters, structure=structure1)
    else:
        api = Cov19API(filters=filters, structure=structure2)
        
    return api.get_json()    

In [35]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api1_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api(1)
    global df
    df=vac_data(apidata)
    vac_graph()

def api2_button_callback(button):
    apidata=access_api(2)
    global df
    df=pcr_data(apidata)
    pcr_graph()

    
btn1_style = wdg.ButtonStyle()    
api1button=wdg.Button(
    description='Vaccinated Data',
    disabled=False,
)
btn1_style.button_color = 'lightblue'
btn1_style.font_weight = 'bold'       
btn1_style.border_color = 'gray' 
btn1_style.border_width = '2px'    
btn1_style.border_radius = '5px' 
api1button.style = btn1_style

btn2_style = wdg.ButtonStyle()
api2button=wdg.Button(
    description='PCR Data',
    disabled=False,
)
btn2_style.button_color = 'lightgreen'
btn2_style.font_weight = 'bold'       
btn2_style.border_color = 'white' 
btn2_style.border_width = '2px'    
btn2_style.border_radius = '5px' 
api2button.style = btn2_style

# remember to register your button callback function with the button
api1button.on_click(api1_button_callback)
api2button.on_click(api2_button_callback)

hbox = wdg.HBox([api1button, api2button])
display(hbox)

# run all cells before clicking on this button

## Graphs and Analysis

The second graph represents the number of vaccinated individuals. It displays data for the first, second, and third doses, as well as the number of deaths.

The first graph shows the changes in the number of PCR tests and confirmed cases.

In [38]:
pcrcols=wdg.SelectMultiple(
    options=['case', 'pcr', 'hospital'], # options available
    value=['case', 'pcr'], # initial value
    rows=3, # rows of the selection box
    description='pcr',
    disabled=False
)

def pcr_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        pcrdf.plot(y=list(graphcolumns))
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(pcr_graph, {'graphcolumns': pcrcols})

display(pcrcols, output)

SelectMultiple(description='pcr', index=(0, 1), options=('case', 'pcr', 'hospital'), rows=3, value=('case', 'p…

Output()

In [39]:
vacdf = pd.read_pickle("vacdf.pkl")

vaccols=wdg.SelectMultiple(
    options=['death', 'first', 'second', 'third'], # options available
    value=['death', 'first', 'second', 'third'], # initial value
    rows=4, # rows of the selection box
    description='vaccination',
    disabled=False
)

def vac_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        vacdf.plot(y=list(graphcolumns))
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(vac_graph, {'graphcolumns': vaccols})

display(vaccols, output)

SelectMultiple(description='vaccination', index=(0, 1, 2, 3), options=('death', 'first', 'second', 'third'), r…

Output()